In [ ]:
pip install reverse_geocoder

In [ ]:
import requests
import pandas as pd
import numpy as np
import time
from datetime import datetime, timedelta
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
from typing import List, Tuple, Optional
from sklearn.svm import SVR
from typing import List, Tuple, Dict
import reverse_geocoder as rg
from sklearn.preprocessing import StandardScaler
import sqlite3
from sqlalchemy import create_engine
from sqlalchemy.types import FLOAT, TEXT, INTEGER, DATE

# Extraction

In [ ]:
# Global configuration
class WeatherConfig:
    API_URL = "https://api.open-meteo.com/v1/forecast"
    HOURLY_VARIABLES = [
        "temperature_2m",
        "relative_humidity_2m",
        "precipitation",
        "wind_speed_10m",
        "weather_code"
    ]
    MAX_DAYS_PER_REQUEST = 90
    RETRY_CONFIG = {
        "total": 3,
        "backoff_factor": 1,
        "status_forcelist": [429, 500, 502, 503, 504],
        "allowed_methods": ["GET"]
    }

In [ ]:
# Configure HTTP session with retry mechanism
def setup_session() -> requests.Session:
    """Set up and return a session with retry configuration."""
    session = requests.Session()
    adapter = HTTPAdapter(max_retries=Retry(**WeatherConfig.RETRY_CONFIG))
    session.mount("https://", adapter)
    return session

In [ ]:
# Construct parameters for API request
def build_api_params(latitude: float, longitude: float, start_date: str, end_date: str) -> dict:
    """Build API request parameters."""
    return {
        "latitude": latitude,
        "longitude": longitude,
        "start_date": start_date,
        "end_date": end_date,
        "hourly": ",".join(WeatherConfig.HOURLY_VARIABLES),
        "timezone": "Europe/Paris"
    }

In [ ]:
# Fetch a single chunk of weather data for a specific period
def fetch_weather_chunk(
    session: requests.Session,
    latitude: float,
    longitude: float,
    start_date: datetime,
    end_date: datetime
) -> Optional[pd.DataFrame]:
    """Fetch a chunk of weather data for a given period."""
    try:
        params = build_api_params(
            latitude,
            longitude,
            start_date.strftime('%Y-%m-%d'),
            end_date.strftime('%Y-%m-%d')
        )
        
        response = session.get(WeatherConfig.API_URL, params=params, timeout=15)
        response.raise_for_status()
        
        data = response.json()
        if "hourly" not in data:
            print(f"No hourly data available for {latitude, longitude}")
            return None
            
        df = pd.DataFrame(data["hourly"])
        df["latitude"] = latitude
        df["longitude"] = longitude
        return df
        
    except requests.exceptions.HTTPError as http_err:
        if response.status_code == 429:
            print(f"Rate limit reached for {latitude, longitude}. Waiting...")
            time.sleep(10)
            return None
        elif response.status_code == 404:
            print(f"Error 404: Invalid URL or parameters for {latitude, longitude}")
            return None
        else:
            print(f"HTTP error for {latitude, longitude}: {str(http_err)}")
            return None
    except requests.exceptions.RequestException as e:
        print(f"Connection error for {latitude, longitude}: {str(e)}")
        return None

In [ ]:
# Collect all weather data for a single location
def fetch_location_data(
    session: requests.Session,
    latitude: float,
    longitude: float,
    needed_hours: int
) -> pd.DataFrame:
    """Fetch all weather data for a given location."""
    all_data = []
    remaining_hours = needed_hours
    end_date = datetime.now()
    
    while remaining_hours > 0:
        chunk_days = min(WeatherConfig.MAX_DAYS_PER_REQUEST, (remaining_hours // 24) + 1)
        start_date = end_date - timedelta(days=chunk_days)
        
        chunk_df = fetch_weather_chunk(session, latitude, longitude, start_date, end_date)
        
        if chunk_df is not None:
            all_data.append(chunk_df)
            fetched_hours = len(chunk_df)
            remaining_hours -= fetched_hours
            print(f"{fetched_hours} hourly records added for {latitude, longitude} (remaining: {remaining_hours})")
        else:
            break
            
        end_date = start_date - timedelta(days=1)
        time.sleep(1)  # Respect API rate limits
        
        if chunk_df is not None and len(chunk_df) == 0:
            break
    
    final_df = pd.concat(all_data, ignore_index=True) if all_data else pd.DataFrame()
    return final_df.iloc[:needed_hours]

In [ ]:
# Orchestrate data collection for multiple locations
def collect_weather_data(
    locations: List[Tuple[float, float]],
    needed_hours: int
) -> pd.DataFrame:
    """Main pipeline to collect weather data for all locations."""
    session = setup_session()
    all_dfs = []
    
    for lat, lon in locations:
        print(f"\n=== Fetching data for location ({lat}, {lon}) ===")
        df_location = fetch_location_data(session, lat, lon, needed_hours)
        
        if not df_location.empty:
            all_dfs.append(df_location)
        
        time.sleep(2)  # Pause between locations
    
    final_df = pd.concat(all_dfs, ignore_index=True) if all_dfs else pd.DataFrame()
    
    # Summary
    if not final_df.empty:
        counts = final_df["latitude"].value_counts()
        print("\n=== Summary ===")
        print(f"Total records retrieved: {len(final_df)}")
        print("Breakdown by location:")
        print(counts.to_string())
    
    return final_df

In [ ]:
# Save DataFrame to CSV file
def save_to_csv(df: pd.DataFrame, filename: str) -> None:
    """Save the DataFrame to a CSV file."""
    df.to_csv(filename, index=False)
    print(f"Data saved to {filename}")

In [ ]:
# Entry point for the weather data pipeline
def main():
    """Main function to run the weather data pipeline."""
    LOCATIONS = [
        (48.8566, 2.3522),  # Paris
        (45.7640, 4.8357),  # Lyon
        (43.2965, 5.3698),  # Marseille
        (44.8378, -0.5792)  # Bordeaux
    ]
    NEEDED_HOURS = 3000
    
    # Run the pipeline
    df_meteo = collect_weather_data(LOCATIONS, NEEDED_HOURS)
    if not df_meteo.empty:
        save_to_csv(df_meteo, "weather_data_extracted.csv")

In [ ]:
if __name__ == "__main__":
    main()

# Cleaning

In [ ]:
# Configuration class for cleaning pipeline settings
class CleaningConfig:
    SEQ_SIZE = 72
    COLS_TO_IMPUTE = ["temperature_2m", "relative_humidity_2m", "precipitation", "wind_speed_10m"]
    SVR_PARAMS = {"kernel": "linear", "gamma": "auto", "C": 1.0, "epsilon": 0.1}

In [ ]:
# Convert time and weather_code columns to appropriate formats
def preprocess_dataframe(df: pd.DataFrame) -> pd.DataFrame:
    """Convert time to datetime and weather_code to category, then sort by time."""
    df_clean = df.copy()
    df_clean["time"] = pd.to_datetime(df_clean["time"], format="%Y-%m-%dT%H:%M")
    df_clean["weather_code"] = df_clean["weather_code"].astype("category")
    df_clean.sort_values(by="time", ascending=False, inplace=True, ignore_index=True)
    return df_clean

In [ ]:
# Filter rows to keep only the first third of rows with missing values
def filter_missing_rows(df: pd.DataFrame) -> pd.DataFrame:
    """Keep only the first third of rows with missing values and all non-missing rows."""
    nan_rows = df[df.isna().any(axis=1)]
    n = len(nan_rows)
    first_third_nan_rows = nan_rows.iloc[:n // 3]
    return pd.concat([
        df[~df.isna().any(axis=1)],  # Rows without NaN
        first_third_nan_rows         # First third of rows with NaN
    ], ignore_index=True)

In [ ]:
# Split dataframe into separate dataframes by location
def split_by_location(df: pd.DataFrame) -> List[pd.DataFrame]:
    """Split dataframe into separate dataframes based on unique latitude-longitude pairs."""
    locs = df[["latitude", "longitude"]].drop_duplicates().values
    return [
        df[(df["latitude"] == lat) & (df["longitude"] == lon)].copy()
        for lat, lon in locs
    ]

In [ ]:
# Convert time series data into sequences for SVR training
def to_sequence(data: pd.Series, seq_size: int = CleaningConfig.SEQ_SIZE) -> Tuple[np.ndarray, np.ndarray]:
    """Convert a time series into sequences for SVR training."""
    x, y = [], []
    data = data.reset_index(drop=True)
    
    for i in range(len(data) - seq_size):
        window = data[i:(i + seq_size)].values
        after_window = data[i + seq_size]
        x.append(window)
        y.append(after_window)
    
    return np.array(x), np.array(y)

In [ ]:
# Train SVR models for numerical columns
def train_svr_models(df: pd.DataFrame, cols_to_impute: List[str]) -> dict:
    """Train SVR models for each numerical column to impute missing values."""
    models = {}
    for col in cols_to_impute:
        print(f"Processing column {col}")
        series = df[col].dropna()
        X, y = to_sequence(series, seq_size=CleaningConfig.SEQ_SIZE)
        
        model = SVR(**CleaningConfig.SVR_PARAMS)
        print("Training model...")
        model.fit(X, y)
        print("Training completed.")
        
        models[col] = model
    
    return models

In [ ]:
# Impute missing values in a dataframe using SVR and mode for weather_code
def impute_dataset(df: pd.DataFrame, cols_to_impute: List[str]) -> pd.DataFrame:
    """Impute numerical columns with SVR models and weather_code with mode."""
    df_imputed = df.copy().reset_index(drop=True)
    models = train_svr_models(df_imputed, cols_to_impute)
    
    for col in cols_to_impute:
        print(f"\n=== Imputing column: {col} ===")
        model = models[col]
        nan_indices = df_imputed[df_imputed[col].isna()].index
        
        for idx in nan_indices:
            start_idx = idx - CleaningConfig.SEQ_SIZE
            if start_idx < 0:
                continue
                
            window = df_imputed.loc[start_idx:idx - 1, col]
            if window.isna().any() or len(window) != CleaningConfig.SEQ_SIZE:
                continue
                
            input_seq = window.values.reshape(1, -1)
            prediction = model.predict(input_seq)[0]
            df_imputed.at[idx, col] = prediction
    
    if "weather_code" in df_imputed.columns:
        mode_weather = df_imputed["weather_code"].mode()[0]
        df_imputed["weather_code"] = df_imputed["weather_code"].fillna(mode_weather)
    
    return df_imputed

In [ ]:
# Process all location dataframes and combine results
def process_locations(dfs: List[pd.DataFrame], cols_to_impute: List[str]) -> pd.DataFrame:
    """Impute missing values for each location dataframe and combine results."""
    imputed_dfs = []
    
    for i, df in enumerate(dfs, start=1):
        print(f"\n=== Processing dataset df_loc{i} ===")
        df_imputed = impute_dataset(df, cols_to_impute)
        imputed_dfs.append(df_imputed)
    
    final_df = pd.concat(imputed_dfs, ignore_index=True)
    final_df.sort_values(by="time", ascending=False, inplace=True, ignore_index=True)
    return final_df

In [ ]:
# Save the final cleaned dataframe to CSV
def save_cleaned_data(df: pd.DataFrame, filename: str) -> None:
    """Save the cleaned dataframe to a CSV file."""
    df.to_csv(filename, index=False)
    print(f"Cleaned data saved to {filename}")

In [ ]:
# Main pipeline for cleaning weather data
def clean_weather_data(input_file: str, output_file: str) -> pd.DataFrame:
    """Execute the complete cleaning pipeline for weather data."""
    # Load data
    df = pd.read_csv(input_file)
    
    # Preprocess data
    df_clean = preprocess_dataframe(df)
    
    # Filter missing rows
    df_filtered = filter_missing_rows(df_clean)
    
    # Split by location
    location_dfs = split_by_location(df_filtered)
    
    # Process and impute all locations
    final_df = process_locations(location_dfs, CleaningConfig.COLS_TO_IMPUTE)
    
    # Save results
    save_cleaned_data(final_df, output_file)
    
    return final_df

In [ ]:
# Entry point for the cleaning pipeline
def main():
    """Run the weather data cleaning pipeline."""
    INPUT_FILE = "weather_data_extracted.csv"
    OUTPUT_FILE = "weather_data_cleaned.csv"
    
    clean_weather_data(INPUT_FILE, OUTPUT_FILE)

In [ ]:
if __name__ == "__main__":
    main()

# Transforming

In [ ]:
# Configuration class for transformation pipeline settings
class TransformConfig:
    COLUMNS_TO_NORMALIZE = ["temperature_2m", "relative_humidity_2m", "precipitation", "wind_speed_10m", "heat_index"]
    TIME_BINS = [0, 6, 12, 18, 24]
    TIME_LABELS = ["Night", "Morning", "Afternoon", "Evening"]
    AGGREGATION_DICT = {
        "temperature_2m": "mean",
        "relative_humidity_2m": "mean",
        "precipitation": "mean",
        "wind_speed_10m": "mean",
        "weather_code": lambda x: x.mode()[0],
        "location": "first"
    }

In [ ]:
# Preprocess dataframe by converting column types
def preprocess_dataframe(df: pd.DataFrame) -> pd.DataFrame:
    """Convert time to datetime and weather_code to category."""
    df_clean = df.copy()
    df_clean["time"] = pd.to_datetime(df_clean["time"], format="%Y-%m-%d %H:%M:%S")
    df_clean["weather_code"] = df_clean["weather_code"].astype("category")
    return df_clean

In [ ]:
# Perform reverse geocoding and create location dataframe
def add_location_data(df: pd.DataFrame) -> pd.DataFrame:
    """Add location names to the dataframe using reverse geocoding."""
    locs = df[["latitude", "longitude"]].drop_duplicates().values
    coordinates = [(lat, lon) for lat, lon in locs]
    results = rg.search(coordinates)
    
    location_data = [
        {"latitude": lat, "longitude": lon, "location": f"{res['name']}"}
        for (lat, lon), res in zip(locs, results)
    ]
    location_df = pd.DataFrame(location_data)
    
    df_with_locations = df.merge(location_df, on=["latitude", "longitude"], how="left")
    df_with_locations["time"] = pd.to_datetime(df_with_locations["time"], format="%Y-%m-%d %H:%M:%S")
    df_with_locations["weather_code"] = df_with_locations["weather_code"].astype("category")
    df_with_locations["location"] = df_with_locations["location"].astype("category")
    
    return df_with_locations

In [ ]:
# Aggregate data by time of day and location
def aggregate_by_time_of_day(df: pd.DataFrame) -> pd.DataFrame:
    """Transform data by aggregating based on time of day and location."""
    df_transformed = df.copy()
    df_transformed["time_of_day"] = pd.cut(
        df_transformed["time"].dt.hour,
        bins=TransformConfig.TIME_BINS,
        labels=TransformConfig.TIME_LABELS,
        right=False,
        include_lowest=True
    )
    df_transformed["date"] = df_transformed["time"].dt.date
    
    result = df_transformed.groupby(
        ["latitude", "longitude", "date", "time_of_day"],
        observed=True
    ).agg(TransformConfig.AGGREGATION_DICT).reset_index()
    
    result["time_of_day"] = result["time_of_day"].astype("category")
    result["weather_code"] = result["weather_code"].astype("category")
    result["location"] = result["location"].astype("category")
    
    return result

In [ ]:
# Calculate heat index for each row
def calculate_heat_index(row: pd.Series) -> float:
    """Calculate heat index based on temperature and relative humidity."""
    T = row["temperature_2m"]
    RH = row["relative_humidity_2m"]
    if T >= 20:
        return T + 0.33 * RH - 0.7
    return T

In [ ]:
# Add heat index and raining indicator columns
def add_derived_features(df: pd.DataFrame) -> pd.DataFrame:
    """Add heat index and is_raining columns to the dataframe."""
    df_features = df.copy()
    df_features["heat_index"] = df_features.apply(calculate_heat_index, axis=1)
    df_features["is_raining"] = (df_features["precipitation"] > 0).astype(int)
    return df_features

In [ ]:
# Normalize specified columns using StandardScaler
def normalize_data(df: pd.DataFrame, columns_to_normalize: List[str]) -> pd.DataFrame:
    """Normalize specified columns using StandardScaler (Z-score)."""
    df_normalized = df.copy()
    scaler = StandardScaler()
    
    missing_cols = [col for col in columns_to_normalize if col not in df_normalized.columns]
    if missing_cols:
        raise ValueError(f"Missing columns: {missing_cols}")
    
    print("\nBefore normalization:")
    print(df_normalized[columns_to_normalize].describe())
    
    df_normalized[columns_to_normalize] = scaler.fit_transform(df_normalized[columns_to_normalize])
    
    print("\nAfter normalization:")
    print(df_normalized[columns_to_normalize].describe())
    
    return df_normalized

In [ ]:
# Save the transformed dataframe to CSV
def save_transformed_data(df: pd.DataFrame, filename: str) -> None:
    """Save the transformed dataframe to a CSV file."""
    df.to_csv(filename, index=False)
    print(f"Transformed data saved to {filename}")

In [ ]:
# Main pipeline for transforming weather data
def transform_weather_data(input_file: str, output_file: str) -> pd.DataFrame:
    """Execute the complete transformation pipeline for weather data."""
    # Load data
    df = pd.read_csv(input_file)
    
    # Preprocess data
    df_clean = preprocess_dataframe(df)
    
    # Add location data
    df_with_locations = add_location_data(df_clean)
    
    # Aggregate by time of day
    df_time_of_day = aggregate_by_time_of_day(df_with_locations)
    
    # Add derived features
    df_features = add_derived_features(df_time_of_day)
    
    # Normalize data
    df_normalized = normalize_data(df_features, TransformConfig.COLUMNS_TO_NORMALIZE)
    
    # Save results
    save_transformed_data(df_normalized, output_file)
    
    return df_normalized

In [ ]:
# Entry point for the transformation pipeline
def main():
    """Run the weather data transformation pipeline."""
    INPUT_FILE = "weather_data_cleaned.csv"
    OUTPUT_FILE = "weather_data_transformed.csv"
    
    transform_weather_data(INPUT_FILE, OUTPUT_FILE)

In [ ]:
if __name__ == "__main__":
    main()

# Loading

In [ ]:
# Configuration class for database pipeline settings
class DatabaseConfig:
    DATABASE_NAME = "weather_database.db"
    LOCATIONS_TABLE = "Locations"
    OBSERVATIONS_TABLE = "Weather_Observations"
    WEATHER_COLUMNS = [
        "location_id", "date", "time_of_day", "temperature_2m", "relative_humidity_2m",
        "precipitation", "wind_speed_10m", "weather_code", "heat_index", "is_raining"
    ]

In [ ]:
# Preprocess dataframe by converting column types
def preprocess_dataframe(df: pd.DataFrame) -> pd.DataFrame:
    """Convert columns to appropriate data types for database storage."""
    df_clean = df.copy()
    df_clean["time_of_day"] = df_clean["time_of_day"].astype("category")
    df_clean["location"] = df_clean["location"].astype("category")
    df_clean["weather_code"] = df_clean["weather_code"].astype(str)  # Convert float64 to string
    df_clean["date"] = pd.to_datetime(df_clean["date"]).dt.date  # Convert to datetime.date
    df_clean["is_raining"] = df_clean["is_raining"].astype("int32")  # Optimize to int32
    return df_clean


In [ ]:
# Create SQLite database tables
def create_database_tables(conn: sqlite3.Connection) -> None:
    """Create Locations and Weather_Observations tables in the SQLite database."""
    cursor = conn.cursor()
    
    cursor.execute(f'''
        CREATE TABLE IF NOT EXISTS {DatabaseConfig.LOCATIONS_TABLE} (
            location_id INTEGER PRIMARY KEY AUTOINCREMENT,
            latitude FLOAT NOT NULL,
            longitude FLOAT NOT NULL,
            location TEXT NOT NULL
        )
    ''')
    
    cursor.execute(f'''
        CREATE TABLE IF NOT EXISTS {DatabaseConfig.OBSERVATIONS_TABLE} (
            observation_id INTEGER PRIMARY KEY AUTOINCREMENT,
            location_id INTEGER NOT NULL,
            date DATE NOT NULL,
            time_of_day TEXT NOT NULL,
            temperature_2m FLOAT NOT NULL,
            relative_humidity_2m FLOAT NOT NULL,
            precipitation FLOAT NOT NULL,
            wind_speed_10m FLOAT NOT NULL,
            weather_code TEXT NOT NULL,
            heat_index FLOAT NOT NULL,
            is_raining INTEGER NOT NULL,
            FOREIGN KEY (location_id) REFERENCES {DatabaseConfig.LOCATIONS_TABLE}(location_id)
        )
    ''')
    
    conn.commit()

In [ ]:
# Insert unique locations into the Locations table
def insert_locations(df: pd.DataFrame, engine: sqlalchemy.engine.Engine) -> Dict[Tuple[float, float], int]:
    """Insert unique locations into the Locations table and return a location map."""
    locations = df[["latitude", "longitude", "location"]].drop_duplicates()
    locations.to_sql(
        DatabaseConfig.LOCATIONS_TABLE,
        engine,
        if_exists="append",
        index=False,
        dtype={"latitude": FLOAT, "longitude": FLOAT, "location": TEXT}
    )
    
    # Create a location map (latitude, longitude) -> location_id
    location_df = pd.read_sql_query(f"SELECT * FROM {DatabaseConfig.LOCATIONS_TABLE}", engine)
    location_map = {
        (row["latitude"], row["longitude"]): row["location_id"]
        for _, row in location_df.iterrows()
    }
    
    return location_map

In [ ]:
# Add location_id to the dataframe
def add_location_id(df: pd.DataFrame, location_map: Dict[Tuple[float, float], int]) -> pd.DataFrame:
    """Add location_id column to the dataframe using the location map."""
    df_with_location_id = df.copy()
    df_with_location_id["location_id"] = df_with_location_id.apply(
        lambda row: location_map.get((row["latitude"], row["longitude"])), axis=1
    )
    return df_with_location_id

In [ ]:
# Insert weather observations into the Weather_Observations table
def insert_weather_observations(df: pd.DataFrame, engine: sqlalchemy.engine.Engine) -> None:
    """Insert weather observations into the Weather_Observations table."""
    df[DatabaseConfig.WEATHER_COLUMNS].to_sql(
        DatabaseConfig.OBSERVATIONS_TABLE,
        engine,
        if_exists="append",
        index=False,
        dtype={
            "location_id": INTEGER,
            "date": DATE,
            "time_of_day": TEXT,
            "temperature_2m": FLOAT,
            "relative_humidity_2m": FLOAT,
            "precipitation": FLOAT,
            "wind_speed_10m": FLOAT,
            "weather_code": TEXT,
            "heat_index": FLOAT,
            "is_raining": INTEGER
        }
    )

In [ ]:
# Display sample data from database tables
def display_sample_data(conn: sqlite3.Connection) -> None:
    """Display the first 5 rows from Locations and Weather_Observations tables."""
    print("Locations:")
    print(pd.read_sql_query(f"SELECT * FROM {DatabaseConfig.LOCATIONS_TABLE} LIMIT 5", conn))
    print("\nWeather Observations:")
    print(pd.read_sql_query(f"SELECT * FROM {DatabaseConfig.OBSERVATIONS_TABLE} LIMIT 5", conn))

In [ ]:
# Main pipeline for storing weather data in SQLite database
def store_weather_data(input_file: str) -> None:
    """Execute the complete pipeline for storing weather data in a SQLite database."""
    # Load data
    df = pd.read_csv(input_file)
    
    # Preprocess data
    df_clean = preprocess_dataframe(df)
    
    # Connect to SQLite database
    conn = sqlite3.connect(DatabaseConfig.DATABASE_NAME)
    engine = create_engine(f"sqlite:///{DatabaseConfig.DATABASE_NAME}")
    
    try:
        # Create tables
        create_database_tables(conn)
        
        # Insert locations and get location map
        location_map = insert_locations(df_clean, engine)
        
        # Add location_id to dataframe
        df_with_location_id = add_location_id(df_clean, location_map)
        
        # Insert weather observations
        insert_weather_observations(df_with_location_id, engine)
        
        # Display sample data
        display_sample_data(conn)
        
    finally:
        conn.close()

In [ ]:
# Entry point for the database storage pipeline
def main():
    """Run the weather data storage pipeline."""
    INPUT_FILE = "weather_data_transformed.csv"
    store_weather_data(INPUT_FILE)

In [ ]:
if __name__ == "__main__":
    main()